In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!unzip "/content/gdrive/MyDrive/2-專案/蘭花辨識比賽/training.zip"

Archive:  /content/gdrive/MyDrive/2-專案/蘭花辨識比賽/training.zip
   creating: training/
  inflating: __MACOSX/._training     
  inflating: training/fl49dcj60u.jpg  
  inflating: __MACOSX/training/._fl49dcj60u.jpg  
  inflating: training/zdse9yhx2f.jpg  
  inflating: __MACOSX/training/._zdse9yhx2f.jpg  
  inflating: training/wg36obamye.jpg  
  inflating: __MACOSX/training/._wg36obamye.jpg  
  inflating: training/pov7d2may6.jpg  
  inflating: __MACOSX/training/._pov7d2may6.jpg  
  inflating: training/xe5f8blhdq.jpg  
  inflating: __MACOSX/training/._xe5f8blhdq.jpg  
  inflating: training/jwpt8ld3n4.jpg  
  inflating: __MACOSX/training/._jwpt8ld3n4.jpg  
  inflating: training/igqykm5l7t.jpg  
  inflating: __MACOSX/training/._igqykm5l7t.jpg  
  inflating: training/ncvzaep06j.jpg  
  inflating: __MACOSX/training/._ncvzaep06j.jpg  
  inflating: training/3f47vrk21p.jpg  
  inflating: __MACOSX/training/._3f47vrk21p.jpg  
  inflating: training/j3ldu4cvz5.jpg  
  inflating: __MACOSX/training/._j3ldu4c

In [7]:
import torch
from torch import nn
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import os
import pandas as pd
import model 
from flower_dataset import FlowerDataset
from trainer_mixup_arcface import Trainer
from config import *

In [8]:
## dataset & dataloader
df_train_label = pd.read_csv('train_label.csv')
df_test_label = pd.read_csv('test_label.csv')

# dataset
train_dataset = FlowerDataset('./training/', df_train_label, image_transform)
test_dataset = FlowerDataset('./training/', df_test_label, image_transform)

# dataloader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [9]:
train_dataset[0][0].shape

torch.Size([3, 448, 448])

In [10]:
# custom layer 
if is_arcface: 
    custom_layer = model.Arcface_Mixup(class_num, embed_size) if is_mixup else model.Arcface(class_num, embed_size)
else: 
    custom_layer = model.FFN(embed_size, class_num)

# final model 
final_model = model.Model(base_model, custom_layer).to(device)

In [ ]:
%%time
train_pipeline = Trainer()
train_pipeline.compile(lr, decay_fn, loss_fn, metric_dict, is_arcface, is_mixup, device=device)
train_pipeline.fit(final_model, train_dataloader, test_dataloader, num_epoch, save_config)

100%|██████████| 55/55 [00:57<00:00,  1.04s/it, loss=5.29]


train_loss: 5.416900591218853
{'Accuracy': 0.1392693966627121, 'CE': 5.09618616104126}


 96%|█████████▋| 53/55 [00:54<00:02,  1.03s/it, loss=5.05]